In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tf-nightly-gpu-2.0-preview
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

!unzip -qq ./gdrive/My\ Drive/YE358311_Fender_apron.zip

Mounted at /content/gdrive


In [0]:
base_dir = "./YE358311_Fender_apron/"

In [8]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True,
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 201 images belonging to 2 classes.
Found 49 images belonging to 2 classes.


In [9]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((32, 224, 224, 3), (32, 2))

In [10]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'YE358311_Healthy': 0, 'YE358311_defects': 1}


In [0]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

In [0]:
base_model.trainable = False

In [0]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(2, activation='softmax')
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 66        
Total params: 2,626,722
Trainable params: 368,738
Non-trainable params: 2,257,984
_________________________________________________________________


In [16]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 4


In [17]:
epochs = 10

history = model.fit(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

Epoch 1/10


W0707 18:34:31.513165 140010291689344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0707 18:34:31.952184 140010291689344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:460: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


7/7 [==============================] - 63s 9s/step - loss: 1.7856 - accuracy: 0.6070 - val_loss: 0.6934 - val_accuracy: 0.4286
Epoch 2/10
7/7 [==============================] - 46s 7s/step - loss: 0.6934 - accuracy: 0.4428 - val_loss: 0.6934 - val_accuracy: 0.4490
Epoch 3/10
7/7 [==============================] - 45s 6s/step - loss: 0.6933 - accuracy: 0.4428 - val_loss: 0.6933 - val_accuracy: 0.4490
Epoch 4/10
7/7 [==============================] - 45s 6s/step - loss: 0.6932 - accuracy: 0.4428 - val_loss: 0.6931 - val_accuracy: 0.5510
Epoch 5/10
7/7 [==============================] - 45s 6s/step - loss: 0.6930 - accuracy: 0.5572 - val_loss: 0.6929 - val_accuracy: 0.5510
Epoch 6/10
7/7 [==============================] - 45s 6s/step - loss: 0.6928 - accuracy: 0.5572 - val_loss: 0.6926 - val_accuracy: 0.5510
Epoch 7/10
7/7 [==============================] - 45s 6s/step - loss: 0.6922 - accuracy: 0.5572 - val_loss: 0.6922 - val_accuracy: 0.5510
Epoch 8/10
7/7 [=============================

In [0]:
base_model.trainable = True

In [19]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  155


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [21]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 60


In [22]:
history_fine = model.fit(train_generator, 
                         epochs=5,
                         validation_data=val_generator)

Epoch 1/5
7/7 [==============================] - 67s 10s/step - loss: 0.6906 - accuracy: 0.5572 - val_loss: 0.6912 - val_accuracy: 0.5510
Epoch 2/5
7/7 [==============================] - 48s 7s/step - loss: 0.6903 - accuracy: 0.5572 - val_loss: 0.6917 - val_accuracy: 0.5510
Epoch 3/5
7/7 [==============================] - 45s 6s/step - loss: 0.6906 - accuracy: 0.5572 - val_loss: 0.6917 - val_accuracy: 0.5510
Epoch 4/5
7/7 [==============================] - 46s 7s/step - loss: 0.6913 - accuracy: 0.5572 - val_loss: 0.6917 - val_accuracy: 0.5510
Epoch 5/5
7/7 [==============================] - 45s 6s/step - loss: 0.6910 - accuracy: 0.5572 - val_loss: 0.6907 - val_accuracy: 0.5510


In [23]:
saved_model_dir = 'save/'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

W0707 18:48:19.206828 140010291689344 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1779: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

In [0]:
tf.saved_model.save?

In [0]:
tf.mo